# Python Programming Challenge

## Poker Hand

In this challenge, we have to determine which kind of Poker combination is present in a deck of 5 cards. Every card is a string containing the card value **with the upper-case initial for face-cards** and the **lower-case initial for the suit**, as seen in the examples below:

> "Ah" ➞ Ace of hearts <br>
> "Ks" ➞ King of spades<br>
> "3d" ➞ Three of diamonds<br>
> "Qc" ➞ Queen of clubs <br>

There are 10 different combinations. Here's the list, in descending order of importance:

| Name            | Description                                         |
|-----------------|-----------------------------------------------------|
| Royal Flush     | A, K, Q, J, 10, all with the same suit.             |
| Straight Flush  | Five cards in sequence, all with the same suit.     |
| Four of a Kind  | Four cards of the same rank.                        |
| Full House      | Three of a Kind with a Pair.                        |
| Flush           | Any five cards of the same suit, not in sequence    |
| Straight        | Five cards in a sequence, but not of the same suit. |
| Three of a Kind | Three cards of the same rank.                       |
| Two Pair        | Two different Pairs.                                |
| Pair            | Two cards of the same rank.                         |
| High Card       | No other valid combination.                         |

---------

#### 1. Given a list `hand` containing five strings being the cards. Implement a function called `poker_hand_ranking` that **returns a string with the name of the highest combination obtained.** According to the table above.

**Examples:**

> poker_hand_ranking(["10h", "Jh", "Qh", "Ah", "Kh"]) ➞ "Royal Flush"<br>
> poker_hand_ranking(["3h", "5h", "Qs", "9h", "Ad"]) ➞ "High Card"<br>
> poker_hand_ranking(["10s", "10c", "8d", "10d", "10h"]) ➞ "Four of a Kind"<br>

### Head

In [95]:
# import libraries
from collections import Counter

In [96]:
# for testing
hand1 = ["10h", "Jh", "Qh", "Ah", "Kh"]
hand2 = ["3h", "5h", "Qs", "9h", "Ad"]
hand3 = ["10s", "10c", "8d", "10d", "10h"]
hand4 = ["6d", "2h", "4c", "5d", "3h"]
hand5 = ["6d", "6h", "4c", "5d", "5h"]
hand6 = ["6d", "3d", "4c", "Ad", "3h"]
hand7 = ["2d", "4c", "2h", "5d", "5h"]

### The Cards

In [97]:
# define the face conversion
face_value = [i for i in range(1,14)]
face_key = ['A'] + [str(i) for i in range(2,11)] + ['J', 'Q', 'K']
face = dict(zip(face_key, face_value))

# define the suit conversion
suit = ['h', # hearts
        's', # spades
        'd', # diamonds
        'c'] # clubs

### Functions

##### The 10-Cards

In [98]:
# evaluate if value is 10
def ten(face):
    return face[:2] == '10'

##### Card Names to Values

In [99]:
def card_values(hand):
    # if the card value is 10, use ten() to pull 2 digits rather than just the first
    values = [card[:2] if ten(card) else card[0] for card in hand]
    
    # return the hand as value pairs of value & keys, respectively
    return [face[value] for value in values], [card[-1] for card in hand]

##### Hand Configurations

In [100]:
######## FLUSHES #########

def flush(hand):
    # convert hand to values and create Counter class;
    # the Counter class counts the argument elements, then returns the results as a key/value pair
    # where the key is the element being counted
    values, suits = card_values(hand)
    suit_count = Counter(suits)
    
    # flush requires that the entire hand is the same suit
    # if the Counter returns a result of 5 for any element,
    # then we can conclude that the hand is at least a flush 
    for key, value in suit_count.items():
        if value == 5 and key in suit:
            return True
        
    return False


######## STRAIGHTS #########

def straight(hand):
    # sort values to facilitate evaluation of straight status
    values, suits = card_values(hand)
    values = sorted(values)
    
    for i, value in enumerate(values):
        # evaluate if value is 1 and continue if next value is either 2 or 10
        # a value of 1 represents an ace, which will be next to a 2 or 10
        # in a sorted list if the hand is a straight that ends or begins with an ace
        if value == 1 and (values[i+1] == 2 or values[i+1] == 10):
            continue
        elif value == 1 and not (values[i+1] == 2 or values[i+1] == 10):
            return False
        
        # evaluate if numbers increase by 1
        # if the loop reaches the end of the list, return True
        # otherwise, return False
        try:
            if value + 1 == values[i+1]:
                continue
            else:
                return False
        except IndexError:
            return True
        

######## ROYALS #########

# this is a permutation of a straight so evaluation of a royal hand is a subset
# of the straight function, which strictly begins with a hand value of 1
def royals(hand):
    return straight(hand) and hand[0] == 1
        
        
######## 4-KINDS #########

# Counter class is necessary here to reveal
# any elements that repeats 4 times
def fourkinds(hand):
    values, suits = card_values(hand)
    value_count = Counter(values)
    
    # catch if any elements repeats 4 times
    for count in value_count.values():
            if count == 4:
                return True
    
    return False


######## TTRIPLES #########

# Counter class is necessary here to reveal
# any elements that repeats 3 times
def triples(hand):
    # convert hand to values
    values, suits = card_values(hand)
    value_count = Counter(values)
    
    # catch if any elements repeats 3 times
    for count in value_count.values():
            if count == 3:
                return True
    
    return False


####### PAIRS ########

# Counter class is necessary here to reveal
# any elements that repeats twice
def pairs(hand):
    # convert hand to values
    values, suits = card_values(hand)
    value_count = Counter(values)
    
    # catch if any elements repeats twice
    for count in value_count.values():
            if count == 2:
                return True
    
    return False

##### Evaluate Hands

In [101]:
# all hand configuration evaluator functions return boolean values, True or False

def poker_hand_ranking(hand):
    
    if royals(hand) and flush(hand):
        return 'royal flush', 7
    
    if straight(hand) and flush(hand):
        return 'straight flush', 6
    
    if fourkinds(hand):
        return 'four of a kind', 5
    
    if triples(hand) and pairs(hand):
        return 'full house', 4
    
    if flush(hand):
        return 'flush', 3
    
    if straight(hand):
        return 'straight', 2
    
    if triples(hand):
        return 'triples', 1
    
    if pairs(hand):
        number, suit = card_values(hand)
        
        # using Counter, determine the 2 most common repeating elements
        # if the 2 most common elements both repeat twice, then the hand is a 2-pair
        if Counter(number).most_common(2)[0][1] == 2 and Counter(number).most_common(2)[1][1] == 2:
            return 'two pair'
        else:
            return 'pair'
    
    return max(hand)

In [102]:
print(poker_hand_ranking(hand1))
print(poker_hand_ranking(hand2))
print(poker_hand_ranking(hand3))
print(poker_hand_ranking(hand4))
print(poker_hand_ranking(hand5))
print(poker_hand_ranking(hand6))
print(poker_hand_ranking(hand7))

straight flush
Qs
four of a kind
straight
two pair
pair
two pair


In [103]:
# answer key
hand1 = ["10h", "Jh", "Qh", "Ah", "Kh"] # straight flush, 10 to A for Hearts
hand2 = ["3h", "5h", "Qs", "9h", "Ad"] # highest card, Qs
hand3 = ["10s", "10c", "8d", "10d", "10h"] # 4 of a kind, 10
hand4 = ["6d", "2h", "4c", "5d", "3h"] # straight, 2 to 6
hand5 = ["6d", "6h", "4c", "5d", "5h"] # two pair, 6 & 5
hand6 = ["6d", "3d", "4c", "Ad", "3h"] # pair, 3
hand7 = ["2d", "4c", "2h", "5d", "5h"] # two pair, 2 & 5

------------
### **Stretch Content**

#### 2.  Implement a function `winner_is` that returns the winner given a dictionary with different players and their hands. 
**Example**

We define dictionary like
```python
round_1 = {"John" = ["10h", "Jh", "Qh", "Ah", "Kh"], 
        "Peter" = ["3h", "5h", "Qs", "9h", "Ad"]}
```

Our function returns the name of the winner:
> winner_is(round_1) --> "John"

One table can have up to 10 players.


In [104]:
round_1 = {"John" : ["10h", "Jh", "Qh", "Ah", "Kh"],
        "Peter" : ["3h", "5h", "Qs", "9h", "Ad"]}

In [105]:
def winner_is(round):
    
    
    for key, value
    
    return winner

SyntaxError: invalid syntax (1751875085.py, line 3)

#### 3. Create a function `distribute_cards` that randomly generates and gives 5 cards to every player given a list of player names.

**Example**

> distribute_cards(["John","Peter"])  -> round_1 = {"John" = ["10h", "Jh", "Qh", "Ah", "Kh"], 
        "Peter" = ["3h", "5h", "Qs", "9h", "Ad"]
}